In [1]:
from zeep import Client
from zeep.wsse.username import UsernameToken
from base64 import b64decode
from zipfile import ZipFile
import pandas as pd
import numpy as np
import csv
import io
import json
import os, time, datetime
from datetime import datetime
from time import perf_counter 

In [2]:
dict_creden_TERYT = {'username' : 'TestPubliczny',
                    'password' : '1234abcd',
                    'wsdl' : 'https://uslugaterytws1test.stat.gov.pl/wsdl/terytws1.wsdl'}

In [4]:
token = UsernameToken(username = dict_creden_TERYT['username'],
                     password = dict_creden_TERYT['password'])

In [5]:
token

In [6]:
client = Client(wsdl = dict_creden_TERYT['wsdl'],
               wsse = token)

In [7]:
client

In [8]:
STATE_DATE = datetime.now() # dzieki temu pobierze ostatni dotepny katalog
catalog = client.service.PobierzKatalogTERC(STATE_DATE)
filename = catalog['nazwa_pliku']
content = catalog['plik_zawartosc']
decoded = b64decode(content)

In [10]:
with open(filename, 'wb') as file:
    file.write(decoded)
    file.close()
# wiemy z dokumentacji, że to zip

In [11]:
#rozpakowujemy
zf = ZipFile(filename, 'r')
zf

<zipfile.ZipFile filename='TERC_Urzedowy_2023-08-25' mode='r'>

In [16]:
# z listy zf pobieramy pierwszy plik
with zf.open(zf.namelist()[1]) as csv_file:
    data = pd.read_csv(csv_file, 
                        sep=';', 
                        quoting=csv.QUOTE_NONE, 
                        encoding='utf-8',
                        dtype={
                            'WOJ' : str,
                            'POW' : str, 
                            'GMI' : str, 
                            'RODZ' : str, 
                            'NAZWA' : str, 
                            'STAN_NA'  : str       
                         })

In [17]:
data

,WOJ,POW,GMI,RODZ,NAZWA,NAZWA_DOD,STAN_NA
0,02,NaN,NaN,NaN,DOLNOŚLĄSKIE,województwo,2023-01-01
1,02,01,NaN,NaN,bolesławiecki,powiat,2023-01-01
2,02,01,01,1,Bolesławiec,gmina miejska,2023-01-01
3,02,01,02,2,Bolesławiec,gmina wiejska,2023-01-01
4,02,01,03,2,Gromadka,gmina wiejska,2023-01-01
...,...,...,...,...,...,...,...
4259,32,61,01,1,Koszalin,gmina miejska,2023-01-01
4260,32,62,NaN,NaN,Szczecin,miasto na prawach powiatu,2023-01-01
4261,32,62,01,1,Szczecin,gmina miejska,2023-01-01
4262,32,63,NaN,NaN,Świnoujście,miasto na prawach powiatu,2023-01-01


In [18]:
data_terc = data.copy()

In [19]:
data_terc = data_terc.replace(np.nan, '')
data_terc

,WOJ,POW,GMI,RODZ,NAZWA,NAZWA_DOD,STAN_NA
0,02,,,,DOLNOŚLĄSKIE,województwo,2023-01-01
1,02,01,,,bolesławiecki,powiat,2023-01-01
2,02,01,01,1,Bolesławiec,gmina miejska,2023-01-01
3,02,01,02,2,Bolesławiec,gmina wiejska,2023-01-01
4,02,01,03,2,Gromadka,gmina wiejska,2023-01-01
...,...,...,...,...,...,...,...
4259,32,61,01,1,Koszalin,gmina miejska,2023-01-01
4260,32,62,,,Szczecin,miasto na prawach powiatu,2023-01-01
4261,32,62,01,1,Szczecin,gmina miejska,2023-01-01
4262,32,63,,,Świnoujście,miasto na prawach powiatu,2023-01-01


In [20]:
voivodeship_df = data_terc[data_terc.NAZWA_DOD == "województwo"].copy().drop(['POW',
                                                                             'GMI', 
                                                                             'RODZ',
                                                                             'NAZWA_DOD',
                                                                             'STAN_NA'], axis=1).reset_index(drop=True)
voivodeship_df

,WOJ,NAZWA
0,02,DOLNOŚLĄSKIE
1,04,KUJAWSKO-POMORSKIE
2,06,LUBELSKIE
3,08,LUBUSKIE
4,10,ŁÓDZKIE
5,12,MAŁOPOLSKIE
6,14,MAZOWIECKIE
7,16,OPOLSKIE
8,18,PODKARPACKIE
9,20,PODLASKIE


In [22]:
for i in np.arange(0,len(voivodeship_df.columns)):
    voivodeship_df[voivodeship_df.columns[i]] = voivodeship_df[voivodeship_df.columns[i]].astype(str)
    voivodeship_df[voivodeship_df.columns[i]] = voivodeship_df[voivodeship_df.columns[i]].str.strip()
    voivodeship_df[voivodeship_df.columns[i]] = voivodeship_df[voivodeship_df.columns[i]].str.lower()
    voivodeship_df[voivodeship_df.columns[i]] = voivodeship_df[voivodeship_df.columns[i]].str.replace('\t', '')

In [23]:
voivodeship_df

,WOJ,NAZWA
0,02,dolnośląskie
1,04,kujawsko-pomorskie
2,06,lubelskie
3,08,lubuskie
4,10,łódzkie
5,12,małopolskie
6,14,mazowieckie
7,16,opolskie
8,18,podkarpackie
9,20,podlaskie


In [24]:
np.arange(0,len(voivodeship_df.columns))

array([0, 1])

In [25]:
voivodeship_df = voivodeship_df.rename(columns = { "WOJ" : "WOJ_KOD",
                                                   "NAZWA" : "WOJ_NAZ"})

In [26]:
voivodeship_df

,WOJ_KOD,WOJ_NAZ
0,02,dolnośląskie
1,04,kujawsko-pomorskie
2,06,lubelskie
3,08,lubuskie
4,10,łódzkie
5,12,małopolskie
6,14,mazowieckie
7,16,opolskie
8,18,podkarpackie
9,20,podlaskie


In [27]:
voivodeship_df = voivodeship_df.sort_values(by = ['WOJ_KOD'])
voivodeship_df

,WOJ_KOD,WOJ_NAZ
0,02,dolnośląskie
1,04,kujawsko-pomorskie
2,06,lubelskie
3,08,lubuskie
4,10,łódzkie
5,12,małopolskie
6,14,mazowieckie
7,16,opolskie
8,18,podkarpackie
9,20,podlaskie


In [28]:
voivodeship_df['WOJ_ID'] = np.arange(0, len(voivodeship_df))
voivodeship_df

,WOJ_KOD,WOJ_NAZ,WOJ_ID
0,02,dolnośląskie,0
1,04,kujawsko-pomorskie,1
2,06,lubelskie,2
3,08,lubuskie,3
4,10,łódzkie,4
5,12,małopolskie,5
6,14,mazowieckie,6
7,16,opolskie,7
8,18,podkarpackie,8
9,20,podlaskie,9


## Weather

In [29]:
import requests
import json
import pandas as pd
import numpy as np

In [38]:
api_key = "95fa5625b9956369459a99659e897da9"
lat = "48.208176"
lon = "16.373819"
url = "https://api.openweathermap.org/data/2.5/onecall?lat=%s&lon=%s&appid=%s&units=metric" % (lat, lon, api_key)

In [39]:
response = requests.get(url)
data = json.loads(response.text)

In [40]:
data

{'cod': 401,
 'message': 'Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.'}

## Yahoo

In [ ]:
import yahoo_fin.stock_info as yh

In [ ]:
response = yh.get_day_gainers()
response

In [53]:
from requests_html import HTMLSession
session = HTMLSession()
r = session.get('https://python.org/')

In [ ]:
from bs4 import BeautifulSoup

In [64]:
pip install BeautifulSoup4

In [60]:
pip install --upgrade pip

                                              0.0/2.1 MB ? eta -:--:--
     -                                        0.1/2.1 MB 2.6 MB/s eta 0:00:01
     ----                                     0.2/2.1 MB 2.8 MB/s eta 0:00:01
     --------                                 0.4/2.1 MB 3.3 MB/s eta 0:00:01
     --------                                 0.5/2.1 MB 2.6 MB/s eta 0:00:01
     -------------                            0.7/2.1 MB 3.3 MB/s eta 0:00:01
     ------------------                       0.9/2.1 MB 3.5 MB/s eta 0:00:01
     -----------------------                  1.2/2.1 MB 3.9 MB/s eta 0:00:01
     ----------------------------             1.5/2.1 MB 4.1 MB/s eta 0:00:01
     ---------------------------------        1.7/2.1 MB 4.3 MB/s eta 0:00:01
     -------------------------------------    2.0/2.1 MB 4.3 MB/s eta 0:00:01
     ---------------------------------------- 2.1/2.1 MB 4.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2